# Brains, Primates and Artifical Intelligence
### Gen Reynolds
<div class="row" display="flex">
    <div class="column" flex="50%">
        <img src="Images/d305ec2a-9f5a-4894-8cd3-a7c43bb0756b-brain-640.jpg"width="25%">
    </div>
    <div class="column" flex="50%">
        <img src="Images/MVIMG_20180630_141951.jpg"width="25%">
    </div>
    

## Learning How to Make A Model Using Kaggle's Deep Learning Tutorial

In [5]:
# Pointing the code towards the files to use for the model training
from os.path import join
#Want to make this read straight from a zip file! Currently have to unzip the file 
image_dir='tutorial_files/indentifying_dog_breeds/train'
# Testing with one image, want to select a number randomn images within the folder
img_paths=[join(image_dir, filename) for filename in
          ['00ba244566e36e0af3d979320fd3017f.jpg']]


#### Creating a function to read the images and prepare them for modelling 

In [4]:
import numpy as np
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

image_size= 224

def img_read_and_prepare(img_paths,img_height=image_size,img_width=image_size):
    imgs=[load_img(img_path,target_size=(img_height,img_width))for img_path in img_paths]
    img_array= np.array([img_to_array(img) for img in imgs])
    return preprocess_input(img_array)

#### Creating tutorial model using pretrained weights to make predictions

In [6]:
from tensorflow.python.keras.applications import ResNet50

my_model = ResNet50(weights='tutorial_files/Resnet_50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
test_data = img_read_and_prepare(img_paths)
preds = my_model.predict(test_data)

ImportError: `load_weights` requires h5py when loading weights from HDF5.